In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.layers import Dense, ReLU, Flatten, LSTM, Input, MaxPooling1D, Lambda, MaxPooling2D
from tensorflow.keras.layers import Input, Embedding, BatchNormalization, Concatenate, Reshape
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dropout, Conv2D
from tensorflow.keras.models import Sequential, Model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from tensorflow.keras import regularizers
from keras.utils import to_categorical
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split

Lấy data từ 2 file X_train Y_train X_test và Y_test từ các file X_train_cleaned Y_train_cleaned X_test_cleaned Y_test_cleaned

In [2]:
X_tr=pd.read_csv('Twitter_2/X_train_cleaned.csv',header=None,encoding='latin1')
Y_tr=pd.read_csv('Twitter_2/Y_train_cleaned.csv',header=None)
X_t=pd.read_csv('Twitter_2/X_test_cleaned.csv',header=None,encoding='latin1')
Y_t=pd.read_csv('Twitter_2/Y_test_cleaned.csv',header=None)
X_train = X_tr[0].astype(str).tolist()
X_test = X_t[0].astype(str).tolist()


Chuẩn bị data để nhét vào mô hình chuẩn bị việc train mô hình

In [3]:
encoder=LabelEncoder()
Y_train=encoder.fit_transform(Y_tr)
Y_test=encoder.transform(Y_t)
tokenizer=Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X_train)
sequences=tokenizer.texts_to_sequences(X_train)
tr_x=pad_sequences(sequences,maxlen=50)
tr_y=to_categorical(Y_train)
sequences=tokenizer.texts_to_sequences(X_test)
t_x=pad_sequences(sequences,maxlen=50)
t_y=to_categorical(Y_test)

c:\Users\ADMIN.LAPTOP-6P42KB5P\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ADMIN.LAPTOP-6P42KB5P\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Tạo mô hình CNN để chạy data

In [1]:
embedding_dim=300
max_len=50
max_word=20000
batchsize=64
epochs=25
def model1():
    inp=Input(shape=(max_len,))

    embed1=Embedding(input_dim=max_word,output_dim=embedding_dim,input_length=max_len)(inp)
    reshape=Reshape((max_len,embedding_dim,1))(embed1)
    conv0=Conv2D(filters=100,kernel_size=(3,embedding_dim),activation='relu',kernel_regularizer=regularizers.l2(3))(reshape)
    conv1=Conv2D(filters=100,kernel_size=(4,embedding_dim),activation='relu',kernel_regularizer=regularizers.l2(3))(reshape)
    conv2=Conv2D(filters=100,kernel_size=(5,embedding_dim),activation='relu',kernel_regularizer=regularizers.l2(3))(reshape)
    pool0=MaxPooling2D(pool_size=(max_len-3+1,1),strides=(1,1),padding='valid')(conv0)
    pool1=MaxPooling2D(pool_size=(max_len-4+1,1),strides=(1,1),padding='valid')(conv1)
    pool2=MaxPooling2D(pool_size=(max_len-5+1,1),strides=(1,1),padding='valid')(conv2)
    merge=Concatenate(axis=1)([pool0,pool1,pool2])
    flat=Flatten()(merge)
    drop=Dropout(0.5)(flat)
    outp=Dense(3,activation='softmax')(drop)
    model=Model(inputs=inp,outputs=outp)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',Precision,Recall])
    return model

Chạy mô hình với data train

In [2]:
model=model1()
model.summary()
history=model.fit(tr_x, tr_y, epochs=epochs, batch_size=batchsize, validation_split=0.2)

NameError: name 'Input' is not defined

Thử dữ liệu với data test

In [ ]:
pred_label=model.predict(t_x)
predicted=np.argmax(pred_label,axis=1)
true=np.argmax(t_y,axis=1)
print(classification_report(true, predicted))
